# Pravopisne greške

## 5.1. Datoteka ngram modela

`read_model()` i `write_model()` su funkcije koje služe za rad s datotekom ngram modela.
U ovom slučaju radi se o datoteci "hr_unigram.txt" koja sadrži unigram model,
odnosno redak u datoteci je riječ i njena frekvencija (broj pojavljivanja)



In [1]:
from test_v05 import *

def read_model(filename):
    """
    ulaz:
    -filename: putanja datoteke
    izlaz:
    -model jezika
    """
    print('*** loading model ' + filename, end=' ')
    txt = open(filename, 'r', encoding='utf8').read().strip()

    model = {}
    for line in txt.split('\n'):
        if not line.startswith('#'):
            line = line.split('\t')
            ngram, freq = line[:-1][0], int(line[-1])
            model[ngram] = freq
    print('FINISHED')
    return model

def write_model(filename, model, comment = None):
    """
    ulaz:
    -filename: putanja datoteke
    -model: model jezika
    -comment: opcioni komentar na početku datoteke
    izlaz:
    zapiše model jezika u datoteku
    svaki red datoteke je oblika
    w1\tw2\tw3\t...\twn\tfreq
    gdje su w1, ..., wn riječi ngrama, a freq frekvencija
    """
    f = open(filename, 'w', encoding='utf8')
    if comment: f.write('# ' + str(comment) + '\n')
    for freq, ngram in sorted(((freq, ngram) for ngram, freq in model.items()), reverse=True):
        f.write('\t'.join(ngram) + '\t' + str(freq) + '\n')
    f.close()
    
unigram = read_model('hr_unigram.txt')
unigram

*** loading model hr_unigram.txt FINISHED


{'prolama': 5,
 'Straight': 7,
 'izražaju': 10,
 'zagrijalo': 7,
 'Sudbenim': 5,
 'Kopru': 122,
 'nekompetentnim': 26,
 'Groznog': 109,
 'srbijansko-crnogorske': 9,
 'oslovljavajući': 8,
 'Lieberman': 67,
 'carinarnici': 273,
 'Su': 57,
 'strpimo': 6,
 'Augijevih': 7,
 'zahvalni': 246,
 'DIJELU': 290,
 'VJEŽBENIKA': 15,
 'Zagorsku': 8,
 'upoznate': 112,
 'Pijući': 5,
 'neurotski': 10,
 'ugrađenu': 70,
 'izbrano': 16,
 'najelitnijoj': 12,
 'obrambenome': 21,
 'osmogodišnju': 35,
 'retrovizor': 101,
 'aktuara': 74,
 'eksponentima': 6,
 'neuredna': 33,
 'suficitu': 8,
 'izobličenih': 5,
 'Monuments': 25,
 'kompaktnih': 17,
 'omekša': 16,
 'frend': 5,
 'Blatnjak': 232,
 'vidim': 3343,
 'raspodijeli': 30,
 'baštu': 7,
 'otpratio': 28,
 'smrznut': 19,
 'Prospera': 37,
 'baranjskom': 108,
 'veteranki': 14,
 'davatelj': 583,
 'usmjeravajući': 33,
 'obespravljenim': 13,
 'jugovci': 12,
 'rezolucije': 995,
 'Chalabija': 11,
 'Gamay': 6,
 'razmješta': 12,
 'nitrat': 24,
 'Švedsko': 12,
 'Fekalije

## 5.2 Minimalna udaljenost od 1

`gen_deletes()`, `gen_inserts()`, `gen_replaces()` i `gen_transposes()` su funkcija koje za zadanu riječ i abecedu generiraju skup riječi koje su za 1 udaljene od zadane riječi temeljem
brisanja, ubacivanja, zamjene i transpozicije (zamjene dva susjedna znaka) u zadanoj riječi.


Na primjer, neka je zadana riječ: "pas" i abeceda "ABC":
* `gen_deletes()` daje: "as", "ps", "pa"
* `gen_inserts()` daje: "Apas", "Bpas", "Cpas", "pAas", "pBas", "pCas", "paAs", "paBs", "paCs", "pasA", "pasB", "pasC"
* `gen_replaces()` daje: "Aas", "Bas", "Cas", "pAs", "pBs", "pCs", "paA", "paB", "paC"
* `gen_transposes()` daje: "aps", "psa"

### Generiranje riječi nastalih brisanjem jednog znaka

Napravi funkciju **gen_deletes()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **word** riječ
* izlaz:
    * skup svih riječi nastalih brisanjem jednog znaka iz zadane riječi.

Na primjer: za riječ "pas" dobiva se {"as", "ps", "pa"}

In [2]:
def gen_deletes(word):
    """
    ulaz:
    -word: riječ
    izlaz:
    -skup svih riječi nastalih brisanjem jednog znaka iz zadane riječi

    Primjer: za riječ "pas" dobiva se {"as", "ps", "pa"}
    """
    return {word[:i] + word[i+1:] for i in range(len(word))}

testname(gen_deletes)
test(gen_deletes, ('pas', ), {'as', 'ps', 'pa'})
test(gen_deletes, ('kuća', ), {'uća', 'kća', 'kua', 'kuć'})


GEN_DELETES
------------------------------------------------------------
OK	gen_deletes('pas',) 
=> {'pa', 'ps', 'as'}
== {'pa', 'ps', 'as'}

OK	gen_deletes('kuća',) 
=> {'kuć', 'uća', 'kća', 'kua'}
== {'kua', 'uća', 'kća', 'kuć'}



### Generiranje riječi nastalih ubacivanjem jednog znaka

Napravi funkciju **gen_inserts()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **word** riječ
    * **alphabet** znakovi abecede (kao string)
* izlaz:
    * skup svih riječi nastalih dodavanjem svakog znaka iz abecede na sva moguća mjesta u riječi

Na primjer: za riječ "pas" i abecedu "abc" dobiva se  
{"apas", "bpas", "cpas", "paas", "pbas", "pcas", "pabs", "pacs", "pasa", "pasb", "pasc"}

In [3]:
def gen_inserts(word, alphabet):
    """
    ulaz:
    -word: riječ
    -alphabet: znakovi abecede (kao string)
    izlaz:
    -skup svih riječi nastalih dodavanjem svakog znaka iz abecede na sva moguća mjesta u riječi

    Primjer: za riječ "pas" i abecedu "abc" dobiva se
    {"apas", "bpas", "cpas", "paas", "pbas", "pcas", "pabs", "pacs", "pasa", "pasb", "pasc"}
    """
    return {word[:i] + letter + word[i:] for i in range(len(word) + 1) for letter in alphabet}

testname(gen_inserts)
test(gen_inserts, ('pas', 'abc'), {'apas', 'bpas', 'cpas', 'paas', 'pbas', 'pcas', 'pabs', 'pacs', 'pasa', 'pasb', 'pasc'})
test(gen_inserts, ('kuća', 'ABC'), {'Akuća', 'Bkuća', 'Ckuća', 'kAuća', 'kBuća', 'kCuća', 'kuAća', 'kuBća', 'kuCća', 'kućAa', 'kućBa', 'kućCa', 'kućaA', 'kućaB', 'kućaC'})



GEN_INSERTS
------------------------------------------------------------
OK	gen_inserts('pas', 'abc') 
=> {'pabs', 'apas', 'bpas', 'pasa', 'pbas', 'cpas', 'pasc', 'pacs', 'paas', 'pasb', 'pcas'}
== {'pabs', 'apas', 'bpas', 'pbas', 'pasa', 'cpas', 'pasc', 'pacs', 'paas', 'pasb', 'pcas'}

OK	gen_inserts('kuća', 'ABC') 
=> {'kuBća', 'Akuća', 'kCuća', 'kAuća', 'kuCća', 'kuAća', 'kućaC', 'kućBa', 'Ckuća', 'kućCa', 'kućAa', 'kBuća', 'kućaB', 'kućaA', 'Bkuća'}
== {'kućaA', 'kuBća', 'Akuća', 'kCuća', 'kAuća', 'kuAća', 'kuCća', 'kućBa', 'Ckuća', 'kućCa', 'kućAa', 'kBuća', 'kućaB', 'kućaC', 'Bkuća'}



### Generiranje riječi nastalih zamjenom jednog znaka

Napravi funkciju **gen_replaces()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **word** riječ
    * **alphabet** znakovi abecede (kao string)
* izlaz:
    * skup svih riječi nastalih zamjenom svakog znaka iz riječi sa svakim znakom iz abecede


Na primjer: za riječ "pas" i abecedu "abc" dobiva se  
{"aas", "bas", "cas", "pas", "pbs", "pcs", "paa", "pab", "pac"}

In [4]:
def gen_replaces(word, alphabet):
    """
    ulaz:
    -word: riječ
    -alphabet: znakovi abecede (kao string)
    izlaz:
    -skup svih riječi nastalih zamjenom svakog znaka iz riječi sa svakim znakom iz abecede

    Primjer: za riječ "pas" i abecedu "abc" dobiva se
    {"aas", "bas", "cas", "pas", "pbs", "pcs", "paa", "pab", "pac"}
    """
    return {word[:i] + letter + word[i+1:] for i in range(len(word)) for letter in alphabet}

testname(gen_replaces)
test(gen_replaces, ('pas', 'abc'), {'aas', 'bas', 'cas', 'pas', 'pbs', 'pcs', 'paa', 'pab', 'pac'})
test(gen_replaces, ('kuća', 'ABC'), {'Auća', 'Buća', 'Cuća', 'kAća', 'kBća', 'kCća', 'kuAa', 'kuBa', 'kuCa', 'kućA', 'kućB', 'kućC'})



GEN_REPLACES
------------------------------------------------------------
OK	gen_replaces('pas', 'abc') 
=> {'aas', 'pas', 'paa', 'pcs', 'pac', 'pab', 'pbs', 'cas', 'bas'}
== {'aas', 'pas', 'paa', 'pcs', 'pac', 'pab', 'pbs', 'cas', 'bas'}

OK	gen_replaces('kuća', 'ABC') 
=> {'kućB', 'kuCa', 'Buća', 'kCća', 'kućC', 'Cuća', 'kućA', 'kuBa', 'kAća', 'kBća', 'Auća', 'kuAa'}
== {'kućB', 'kuCa', 'Buća', 'kCća', 'kućC', 'Cuća', 'kućA', 'kuBa', 'kAća', 'kBća', 'Auća', 'kuAa'}



### Generiranje riječi nastalih zamjenom mjesta dva susjedna znaka

Napravi funkciju **gen_transposes()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **word** riječ
* izlaz:
    * skup riječi nastalih zamjenom dva susjedna znaka iz zadane riječi


Na primjer: za riječ dobiva se  
{"aps", "psa"}

In [5]:
def gen_transposes(word):
    """
    ulaz:
    -word: riječ
    izlaz:
    -skup riječi nastalih zamjenom dva susjedna znaka iz zadane riječi

    Primjer: za riječ "pas" dobiva se
    {"aps", "psa"}
    """
    return {word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word)-1)}

testname(gen_transposes)
test(gen_transposes, ('pas', ), {'aps', 'psa'})
test(gen_transposes, ('kuća', ), {'ukća', 'kćua', 'kuać'})



GEN_TRANSPOSES
------------------------------------------------------------
OK	gen_transposes('pas',) 
=> {'aps', 'psa'}
== {'aps', 'psa'}

OK	gen_transposes('kuća',) 
=> {'kćua', 'kuać', 'ukća'}
== {'kćua', 'kuać', 'ukća'}



### Generiranje riječi udaljenosti 1

Napravi funkciju **gen_edits1()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **word** riječ
    * **alphabet** abeceda
* izlaz:
    * skup riječi koje su za 1 udaljene od dane riječi
    
Napomena: rezultat je unija skupova dobivenim brisanjem, ubacivanjem, zamjenom i transpozicijom


In [6]:
def gen_edits1(word, alphabet):
    """
    ulaz:
    -word: riječ
    -alphabet: abeceda
    izlaz:
    -skup riječi koji su za 1 udaljeni od dane riječi
    ovisnost:
    -gen_deletes, gen_inserts, gen_replaces, gen_transposes

    Napomena: Rezultat je unija skupova dobivenim brisanjem, ubacivanjem, zamjenom i transpozicijom
    """
    return gen_deletes(word) | gen_inserts(word, alphabet) | gen_replaces(word, alphabet) | gen_transposes(word)

testname(gen_edits1)
test(gen_edits1, ('pas', 'abc'), {'pabs', 'pa', 'pasa', 'pasc', 'paa', 'pab', 'pas', 'bpas', 'apas', 'aps', 'pcs', 'pbas', 'cas', 'pac', 'ps', 'psa', 'cpas', 'pcas', 'aas', 'pbs', 'paas', 'as', 'bas', 'pasb', 'pacs'})



GEN_EDITS1
------------------------------------------------------------
OK	gen_edits1('pas', 'abc') 
=> {'aas', 'pabs', 'apas', 'paa', 'bpas', 'pbas', 'cpas', 'bas', 'pas', 'psa', 'pcs', 'pasa', 'pa', 'ps', 'aps', 'pbs', 'paas', 'pasb', 'pcas', 'pac', 'pab', 'cas', 'pasc', 'pacs', 'as'}
== {'aas', 'pabs', 'apas', 'paa', 'bpas', 'pbas', 'cpas', 'bas', 'pas', 'psa', 'pcs', 'pa', 'pasa', 'ps', 'aps', 'pbs', 'paas', 'pasb', 'pcas', 'pac', 'pab', 'cas', 'pasc', 'pacs', 'as'}



## 5.3 Ispravljanje pravopisnih grešaka

### Kandidati za pravopisnu grešku


Napravi funkciju **spell_candidates()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **word** riječ
    * **alphabet** abeceda
    * **model** unigram model jezika
* izlaz:
    * lista kandidata nastalih presjekom skupa riječi koje su za 1 udaljene od zadane riječi i skupa svih riječi iz unigrama sortiranih po frekvenciji riječi od najveće prema najmanjoj.



In [7]:
def spell_candidates(word, alphabet, model):
    """
    ulaz:
    -word: riječ
    -alphabet: abeceda
    -model: unigram model jezika
    izlaz:
    -lista kandidata nastalih presjekom skupa riječi koje su za 1 udaljene od zadane riječi i
    skupa svih riječi iz unigrama sortiranih po frekvenciji riječi od najveće prema najmanjom
    ovisnost:
    -gen_edits1
    """
    candidates = gen_edits1(word, alphabet) & set(model)
    return sorted(candidates, key = model.get, reverse = True)


testname(spell_candidates)
test(spell_candidates, ('moelkula', 'abcčćdđefghijklmnoprsštuvzž', unigram), ['molekula'])
test(spell_candidates, ('atmo', 'abcčćdđefghijklmnoprsštuvzž', unigram), ['tamo', 'amo', 'ajmo', 'atom', 'ato'])
test(spell_candidates, ('pivt', 'abcčćdđefghijklmnoprsštuvzž', unigram), ['piva', 'pivo', 'pivu', 'pive', 'pit', 'pivot', 'pivat'])



SPELL_CANDIDATES
------------------------------------------------------------
OK	spell_candidates('moelkula', 'abcčćdđefghijklmnoprsštuvzž', {'A-11': 5, 'A-1': 118, 'A-': 5, 'A-10': 52, 'A': 57324}) 
=> ['molekula']
== ['molekula']

OK	spell_candidates('atmo', 'abcčćdđefghijklmnoprsštuvzž', {'A-11': 5, 'A-1': 118, 'A-': 5, 'A-10': 52, 'A': 57324}) 
=> ['tamo', 'amo', 'ajmo', 'atom', 'ato']
== ['tamo', 'amo', 'ajmo', 'atom', 'ato']

OK	spell_candidates('pivt', 'abcčćdđefghijklmnoprsštuvzž', {'A-11': 5, 'A-1': 118, 'A-': 5, 'A-10': 52, 'A': 57324}) 
=> ['piva', 'pivo', 'pivu', 'pive', 'pit', 'pivot', 'pivat']
== ['piva', 'pivo', 'pivu', 'pive', 'pit', 'pivot', 'pivat']

